In [1]:
from operator import add
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

In [2]:
# Download the files from AWS 

!wget -q  https://s3.amazonaws.com/metcs777/flights.csv.bz2
    
!wget -q  https://s3.amazonaws.com/metcs777/airports.csv.bz2

In [3]:
lines = sc.textFile("flights.csv.bz2")
airports_lines = sc.textFile('airports.csv.bz2')


In [5]:
# Removing the Header Line from CSV file
linesHeader = lines.first()
#header = sc.parallelize([linesHeader])
#linesWithOutHeader = lines.subtract(header)

flights_raw = lines\
    .zipWithIndex()\
    .filter(lambda x: x[1] > 2)\
    .keys()

#flishgts = flights_lines\
#    .map(lambda x: x.split(','))
#flights = flights_raw.map(lambda p: (p[0], p[1] , p[2] , p[3], p[4] , p[5] , p[6], p[7] , p[8] , p[9], p[10], '0' if p[11] == '' else p[11], p[24] ))


flights = flights_raw\
    .map(lambda x: x.split(','))\
    .map(lambda x: (x[0], x[1], x[2], x[3], x[4], 
                    x[5], x[6], x[7], x[8], x[9], 
                    x[10], 0 if x[11]=='' else float(x[11]), x[12], x[13], x[14], 
                    x[15], x[16], x[17], x[18], x[19], 
                    x[20], x[21], x[22], x[23], x[24],
                    x[25], x[26], x[27], x[28], x[29], x[30]
))


airports_raw = airports_lines\
    .zipWithIndex()\
    .filter(lambda x:x[1]>2)\
    .keys()

airports = airports_raw\
    .map(lambda x: x.split(','))\
    .map(lambda x: (x[0], x[1], x[2], x[3]))\
    .filter(lambda x: x[0])


mainFlightsData = flights.map(lambda p: (p[0], p[1], p[2], p[3], p[4], p[5],
                                         p[6], p[7], p[8], p[9], p[10], p[11], p[24]))

mainFlightsData.take(1)

[('2015',
  '1',
  '1',
  '4',
  'US',
  '840',
  'N171US',
  'SFO',
  'CLT',
  '0020',
  '0018',
  -2.0,
  '0')]

In [6]:
# number 6 is ORIGIN_AIRPORT
flightsFromBoston = mainFlightsData\
    .filter(lambda p: True if p[7] == "BOS" else False )

# Get the total number of Flights from BOS
# 107847 flights from Logan Airport in Boston
print(flightsFromBoston.count())

# Find the subset of flights departing on the weekend.
weekEndFlights = flightsFromBoston\
    .filter(lambda p: True if (int(p[3])==6 or int(p[3]) ==7) else False)
print(weekEndFlights.count()) # 26092

107847
26092


In [7]:
#Q1 Find a list of Origin Airports
result = flights\
    .map(lambda x:x[7])\
    .distinct()\
    .collect()

for idx, i in enumerate(sorted(result), start=1):
    print("{}: {}".format(idx, i))



1: 10135
2: 10136
3: 10140
4: 10141
5: 10146
6: 10154
7: 10155
8: 10157
9: 10158
10: 10165
11: 10170
12: 10185
13: 10208
14: 10257
15: 10268
16: 10279
17: 10299
18: 10333
19: 10372
20: 10397
21: 10408
22: 10423
23: 10431
24: 10434
25: 10469
26: 10529
27: 10551
28: 10561
29: 10577
30: 10581
31: 10599
32: 10620
33: 10627
34: 10631
35: 10685
36: 10693
37: 10713
38: 10721
39: 10728
40: 10731
41: 10732
42: 10739
43: 10747
44: 10754
45: 10779
46: 10781
47: 10785
48: 10792
49: 10800
50: 10821
51: 10849
52: 10868
53: 10874
54: 10918
55: 10926
56: 10980
57: 10990
58: 10994
59: 11003
60: 11013
61: 11042
62: 11049
63: 11057
64: 11066
65: 11067
66: 11076
67: 11097
68: 11109
69: 11111
70: 11122
71: 11140
72: 11146
73: 11150
74: 11193
75: 11203
76: 11252
77: 11259
78: 11267
79: 11274
80: 11278
81: 11292
82: 11298
83: 11308
84: 11315
85: 11337
86: 11413
87: 11423
88: 11433
89: 11447
90: 11471
91: 11481
92: 11503
93: 11525
94: 11537
95: 11540
96: 11577
97: 11587
98: 11603
99: 11612
100: 11617
101: 116

In [8]:
#Q2 Find a list of (Origin, Destination) pairs
result = flights\
    .map(lambda x: (x[7],x[8]))\
    .distinct()\
    .collect()

for idx, i in enumerate(result, start=1):
    print("{}: {}".format(idx, i))


1: ('SEA', 'ANC')
2: ('LAS', 'ATL')
3: ('ANC', 'SEA')
4: ('ANC', 'PDX')
5: ('PDX', 'MSP')
6: ('SJU', 'MCO')
7: ('HNL', 'KOA')
8: ('IAH', 'MIA')
9: ('HNL', 'LIH')
10: ('SEA', 'DEN')
11: ('ORD', 'FLL')
12: ('RDD', 'SFO')
13: ('BUF', 'JFK')
14: ('ORD', 'DTW')
15: ('SMF', 'DEN')
16: ('AUS', 'LAX')
17: ('BOS', 'BWI')
18: ('PHX', 'DTW')
19: ('MFR', 'DEN')
20: ('SEA', 'IAH')
21: ('BOS', 'PBI')
22: ('BOS', 'JFK')
23: ('JFK', 'BQN')
24: ('CHS', 'EWR')
25: ('CHS', 'DFW')
26: ('BOS', 'BUF')
27: ('PDX', 'EWR')
28: ('IND', 'PHX')
29: ('BOS', 'EWR')
30: ('MCI', 'PHX')
31: ('MCI', 'IAH')
32: ('BDL', 'IAD')
33: ('RDM', 'DEN')
34: ('CID', 'ATL')
35: ('MCI', 'ORD')
36: ('ORD', 'LGA')
37: ('OAK', 'PHX')
38: ('PIT', 'IAH')
39: ('PHX', 'SLC')
40: ('SFO', 'LAX')
41: ('LAX', 'ORD')
42: ('IAH', 'PHX')
43: ('BWI', 'MIA')
44: ('TPA', 'DFW')
45: ('FAI', 'ANC')
46: ('PDX', 'SJC')
47: ('PBI', 'JFK')
48: ('AUS', 'JFK')
49: ('DCA', 'BOS')
50: ('LGA', 'DTW')
51: ('RNO', 'SLC')
52: ('ORD', 'ATL')
53: ('MEM', 'EWR')
54

4468: ('15048', '13930')
4469: ('13931', '11618')
4470: ('10185', '10397')
4471: ('12339', '13198')
4472: ('13232', '11618')
4473: ('13487', '13232')
4474: ('14307', '11057')
4475: ('14027', '12953')
4476: ('14107', '11618')
4477: ('12888', '11292')
4478: ('12217', '12266')
4479: ('11618', '12264')
4480: ('10721', '14771')
4481: ('14492', '12266')
4482: ('10821', '10721')
4483: ('12391', '10821')
4484: ('11259', '12892')
4485: ('11298', '13303')
4486: ('11648', '14869')
4487: ('11603', '14869')
4488: ('12758', '12173')
4489: ('14122', '10397')
4490: ('14679', '14771')
4491: ('10397', '12889')
4492: ('13204', '14100')
4493: ('14570', '14107')
4494: ('12889', '11433')
4495: ('14679', '10397')
4496: ('11775', '10397')
4497: ('11109', '12266')
4498: ('13198', '11433')
4499: ('12982', '12173')
4500: ('10721', '12191')
4501: ('13891', '13232')
4502: ('12191', '11292')
4503: ('14831', '14107')
4504: ('14698', '14107')
4505: ('10713', '14747')
4506: ('14635', '12266')
4507: ('10140', '11298')


In [9]:
#Q3 Find the Origin airport which had the largest departure delay in the month of January
result = flights\
    .filter(lambda x: x[1]=="1")\
    .map(lambda x: (x[7], x[11]) )\
    .top(1, lambda x:x[1])

print(result)

[('BHM', 1988.0)]


In [10]:
#Q4 Find out which carrier has the largest delay on Weekends. 
result = flights\
    .filter(lambda x: x[3]=="6" or x[3]=="7")\
    .map(lambda x: (x[4],x[11]) )\
    .top(1,lambda x: x[1])

print(result)

[('AA', 1878.0)]


In [11]:
#Q5 Which airport has the most cancellation of flights?
result = flights\
    .filter(lambda x: x[24]=="1")\
    .map(lambda x: (x[7],1))\
    .reduceByKey(add)\
    .top(1, lambda x: x[1])

print(result)

[('ORD', 8548)]


In [12]:
#Q6 Find the percent of flights cancelled for each carrier.
result = flights\
    .map(lambda x: (x[4], ((int(x[24])), 1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
    .map(lambda x: (x[0], x[1][0]/x[1][1] * 100))\
    .collect()

for idx, i in enumerate(result, start=1):
    print("{}: {}".format(idx, i))


1: ('AA', 1.5040297086846388)
2: ('NK', 1.7072900604026275)
3: ('UA', 1.274521400053905)
4: ('B6', 1.6012102693148795)
5: ('MQ', 5.0995818512585185)
6: ('F9', 0.6473204456382932)
7: ('WN', 1.2713822111098343)
8: ('OO', 1.6928612584621818)
9: ('EV', 2.6628693111785964)
10: ('US', 2.0466497244797823)
11: ('DL', 0.4365889886868193)
12: ('HA', 0.22419760855884205)
13: ('AS', 0.38778112682587523)
14: ('VX', 0.8626399366751207)


In [13]:
#Q7 Find the largest departure delay for each carrier
result = flights\
    .map(lambda x: (x[4], x[11]) )\
    .reduceByKey(max)\
    .collect()

for idx, i in enumerate(result, start=1):
    print("{}: {}".format(idx, i))


1: ('AA', 1988.0)
2: ('NK', 836.0)
3: ('UA', 1314.0)
4: ('B6', 1006.0)
5: ('MQ', 1544.0)
6: ('F9', 1112.0)
7: ('WN', 665.0)
8: ('OO', 1378.0)
9: ('EV', 1274.0)
10: ('US', 759.0)
11: ('DL', 1289.0)
12: ('HA', 1433.0)
13: ('AS', 963.0)
14: ('VX', 644.0)


In [14]:
#Q8 Find the largest departure delay for each carrier for each month
result = flights\
    .map(lambda x: ((x[4], int(x[1])), x[11]) )\
    .reduceByKey(max)\
    .collect()

for idx, i in enumerate(result, start=1):
    print("{}: {}".format(idx, i))


1: (('UA', 1), 886.0)
2: (('F9', 1), 696.0)
3: (('OO', 2), 913.0)
4: (('MQ', 2), 1154.0)
5: (('NK', 3), 745.0)
6: (('UA', 3), 649.0)
7: (('HA', 3), 952.0)
8: (('NK', 4), 836.0)
9: (('MQ', 4), 973.0)
10: (('OO', 4), 878.0)
11: (('AA', 5), 1492.0)
12: (('AS', 5), 612.0)
13: (('NK', 6), 696.0)
14: (('AA', 7), 1625.0)
15: (('AS', 7), 830.0)
16: (('F9', 7), 793.0)
17: (('DL', 8), 1207.0)
18: (('B6', 8), 522.0)
19: (('F9', 8), 726.0)
20: (('WN', 8), 620.0)
21: (('VX', 9), 331.0)
22: (('EV', 9), 1000.0)
23: (('DL', 10), 1120.0)
24: (('AS', 10), 580.0)
25: (('F9', 10), 550.0)
26: (('EV', 11), 993.0)
27: (('VX', 11), 288.0)
28: (('UA', 12), 1194.0)
29: (('AS', 12), 803.0)
30: (('HA', 12), 1095.0)
31: (('EV', 12), 1274.0)
32: (('US', 1), 638.0)
33: (('B6', 1), 500.0)
34: (('EV', 1), 726.0)
35: (('AS', 2), 735.0)
36: (('DL', 2), 1159.0)
37: (('B6', 3), 674.0)
38: (('EV', 3), 583.0)
39: (('AS', 4), 812.0)
40: (('US', 4), 522.0)
41: (('B6', 4), 500.0)
42: (('HA', 4), 865.0)
43: (('EV', 4), 757.0)
4

In [15]:
#Q9 For each carrier find the average Departure delay
result = flights \
    .map(lambda x: (x[4], (x[11], 1))) \
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
    .map(lambda x: (x[0], x[1][0]/x[1][1])) \
    .collect()

for idx, i in enumerate(result, start=1):
    print("{}: {}".format(idx, i))


1: ('AA', 8.773542906652084)
2: ('NK', 15.68327383944317)
3: ('UA', 14.26220664969373)
4: ('B6', 11.333044995656211)
5: ('MQ', 9.63204268375465)
6: ('F9', 13.27060856928971)
7: ('WN', 10.450107183471951)
8: ('OO', 7.67823058605973)
9: ('EV', 8.49219111957299)
10: ('US', 6.0209194071912036)
11: ('DL', 7.338090448359994)
12: ('HA', 0.48473882945248586)
13: ('AS', 1.7795501970785996)
14: ('VX', 8.947094648078446)


In [16]:
#Q10 For each carrier find the average Departure delay for each month
result = flights.map(lambda x: ((x[4], int(x[1])),(x[11],1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
    .map(lambda x: (x[0], x[1][0]/x[1][1]))\
    .collect()

for idx, i in enumerate(result, start=1):
    print("{}: {}".format(idx, i))


1: (('UA', 1), 13.666987889048054)
2: (('F9', 1), 17.763215697759556)
3: (('OO', 2), 10.620837027438677)
4: (('MQ', 2), 15.991388270230141)
5: (('NK', 3), 12.432021276595744)
6: (('UA', 3), 13.767470128202188)
7: (('HA', 3), 0.8225883098368446)
8: (('NK', 4), 9.86573294018534)
9: (('MQ', 4), 8.615839657520919)
10: (('OO', 4), 5.276855399460763)
11: (('AA', 5), 10.904853500335495)
12: (('AS', 5), -0.19656722517368205)
13: (('NK', 6), 34.43944636678201)
14: (('AA', 7), 9.661222585160989)
15: (('AS', 7), 2.5547689779407117)
16: (('F9', 7), 13.785784919653894)
17: (('DL', 8), 7.74069452851866)
18: (('B6', 8), 14.453034500125913)
19: (('F9', 8), 8.823262097764678)
20: (('WN', 8), 11.626461697741705)
21: (('VX', 9), 3.2865735351183547)
22: (('EV', 9), 3.564671439354086)
23: (('DL', 10), 3.2151895383312157)
24: (('AS', 10), -0.08820073270201148)
25: (('F9', 10), 4.025058634736452)
26: (('EV', 11), 7.015785023019825)
27: (('VX', 11), 5.7207240487624675)
28: (('UA', 12), 17.51819625716456)
29: 

In [17]:
#Q11 Which date of year has the highest rate  of flight cancellations?
# Rate of flight cancellation is calculated by deviding number of canceled flights by total number of flights.
result = flights\
    .map(lambda x: ((x[0],x[1],x[2]), ((int(x[24])), 1)) )\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
    .map(lambda x: (x[0], x[1][0]/x[1][1]))\
    .top(1, lambda x:x[1])

print(result)


[(('2015', '1', '27'), 0.19030023094688223)]


In [18]:
#Q12 Calculate the number of flights to each destination state
# For each carrier, for which state do they have the largest average delay? 
# You will need the airline and airport data sets for this question.

result = flights\
    .map(lambda x: ( (x[4],x[8]), (x[11],1) ))\
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
    .map(lambda x: (x[0][1], (x[0][0], x[1][0]/x[1][1]) ))\
    .leftOuterJoin(airports.map(lambda x: (x[0], x[3])))\
    .filter(lambda x: x[1][1] is not None)\
    .map(lambda x: (x[1][0][0], (x[1][1], x[1][0][1])))\
    .reduceByKey(lambda x,y: (x[0],x[1]) if x[1] > y[1] else (y[0],y[1]) )\
    .map(lambda x: (x[0], x[1][0]))\
    .collect()

for idx, i in enumerate(result, start=1):
    print("{}: {}".format(idx, i))


1: ('NK', 'PA')
2: ('HA', 'NY')
3: ('WN', 'NJ')
4: ('F9', 'WA')
5: ('AS', 'TX')
6: ('AA', 'AR')
7: ('UA', 'AK')
8: ('US', 'FL')
9: ('DL', 'CO')
10: ('MQ', 'MN')
11: ('EV', 'FL')
12: ('VX', 'NY')
13: ('B6', 'NY')
14: ('OO', 'TX')
